In [18]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

# Load the data
# List of Excel files (update paths as needed)
files = [
    "data/patient-medical-record-24.xlsx",
    "data/patient-medical-record-23.xlsx",
    "data/patient-medical-record-18.xlsx",
    "data/patient-medical-record-17.xlsx",
    "data/patient-medical-record-16.xlsx",
    "data/patient-medical-record-15.xlsx",
    "data/patient-medical-record-14.xlsx"
]

# Read and concatenate all DataFrames, keeping only the columns we need
columns_to_keep = ["patients_common_name", "patients_disposition", "patients_days_in_care"]
df_list = []
for file in files:
    temp_df = pd.read_excel(file)
    # Keep only the columns we need if they exist in the file
    available_cols = [col for col in columns_to_keep if col in temp_df.columns]
    df_list.append(temp_df[available_cols])

df = pd.concat(df_list, ignore_index=True)

# df = pd.read_excel("data/patient-medical-record-2017.xlsx")

# Drop rows with missing data
df = df.dropna(subset=["patients_common_name", "patients_disposition"])

# Standardize species names
df["patients_common_name"] = df["patients_common_name"].replace({
    "Douglas's Squirrel": "Douglas Squirrel",
    "Douglas' Squirrel": "Douglas Squirrel"
})

df.to_csv("14to24/release_rate_14to24.csv", index=False)

# PSWC Release Data Analysis (2014-2024)

## Plot all Release Rate

In [21]:
# Group by species and calculate total and released counts
grouped = df.groupby("patients_common_name")["patients_disposition"].value_counts().unstack().fillna(0)

# Calculate total excluding "Dead on arrival" cases
exclude_cols = ["Dead on arrival"] if "Dead on arrival" in grouped.columns else []
included_cols = [col for col in grouped.columns if col not in exclude_cols]
grouped["total_excluding_doa"] = grouped[included_cols].sum(axis=1)
grouped["doa_count"] = grouped.get("Dead on arrival", 0)

grouped["released"] = grouped.get("Released", 0)
grouped["release_rate"] = grouped["released"] / grouped["total_excluding_doa"]
grouped = grouped.reset_index()

# Filter out species with 0% or 100% release rate
grouped = grouped[(grouped["release_rate"] > 0) & (grouped["release_rate"] < 1)]

# Sort by release rate (ascending for horizontal bar chart to show highest at top)
grouped = grouped.sort_values(by="release_rate", ascending=True)

# Create horizontal bar chart for better readability
fig = px.bar(
    grouped,
    y="patients_common_name",
    x="release_rate",
    orientation='h',
    hover_data={"released": ":.0f", "total_excluding_doa": ":.0f", "doa_count": ":.0f"},
    labels={
        "patients_common_name": "Animal Common Name", 
        "release_rate": "Release Rate",
        "released": "Released",
        "total_excluding_doa": "Total Received (excl. DOA)",
        "doa_count": "Dead on Arrival"
    },
    title="Release Rate by Animal Common Name (2014-2024, excluding DOA)",
    height=max(400, len(grouped) * 25)  # Dynamic height based on number of species
)

# Customize hover template with background color
fig.update_traces(
    hovertemplate='<b>%{y}</b><br>' +
                  'Release Rate: %{x:.1%}<br>' +
                  'Released: %{customdata[0]:.0f}<br>' +
                  'Total Received (excl. DOA): %{customdata[1]:.0f}<br>' +
                  'Total DOA: %{customdata[2]:.0f}' +
                  '<extra></extra>'
)

fig.update_layout(
    yaxis={'categoryorder':'total ascending'},  # Ensures bars are sorted by value
    margin=dict(l=200),  # More left margin for animal names
    hoverlabel=dict(
        bgcolor="white",  # White background
        font_size=14,
        font_family="Arial",
        font_color="black",  # Black text
        bordercolor="black"  # Black border for definition
    )
)
fig.show()
fig.write_html("14to24/release_rate.html")

## Species with 0% and 100% Release Rates

In [22]:
# Group by species and calculate total and released counts (for 0% and 100% rates)
grouped_all = df.groupby("patients_common_name")["patients_disposition"].value_counts().unstack().fillna(0)

# Calculate total excluding "Dead on arrival" cases
exclude_cols = ["Dead on arrival"] if "Dead on arrival" in grouped_all.columns else []
included_cols = [col for col in grouped_all.columns if col not in exclude_cols]
grouped_all["total_excluding_doa"] = grouped_all[included_cols].sum(axis=1)
grouped_all["doa_count"] = grouped_all.get("Dead on arrival", 0)
grouped_all["released"] = grouped_all.get("Released", 0)
grouped_all["release_rate"] = grouped_all["released"] / grouped_all["total_excluding_doa"]
grouped_all = grouped_all.reset_index()

# Filter for species with 0% release rate
zero_release = grouped_all[grouped_all["release_rate"] == 0][
    ["patients_common_name", "released", "total_excluding_doa", "doa_count"]
].copy()
zero_release.columns = ["Common Name", "Released", "Total Received (excl. DOA)", "Dead on Arrival"]
zero_release = zero_release.sort_values("Total Received (excl. DOA)", ascending=False)

# Filter for species with 100% release rate
hundred_release = grouped_all[grouped_all["release_rate"] == 1][
    ["patients_common_name", "released", "total_excluding_doa", "doa_count"]
].copy()
hundred_release.columns = ["Common Name", "Released", "Total Received (excl. DOA)", "Dead on Arrival"]
hundred_release = hundred_release.sort_values("Released", ascending=False)

# Display tables
print("Species with 0% Release Rate:")
print("-" * 80)
if len(zero_release) > 0:
    print(zero_release.to_string(index=False))
else:
    print("No species with 0% release rate")

print("\n")
print("Species with 100% Release Rate:")
print("-" * 80)
if len(hundred_release) > 0:
    print(hundred_release.to_string(index=False))
else:
    print("No species with 100% release rate")

Species with 0% Release Rate:
--------------------------------------------------------------------------------
               Common Name  Released  Total Received (excl. DOA)  Dead on Arrival
               Winter Wren       0.0                        14.0              4.0
                   Bushtit       0.0                        14.0              3.0
         Belted Kingfisher       0.0                        10.0              0.0
    Golden-crowned Kinglet       0.0                        10.0              5.0
                House Wren       0.0                         7.0              0.0
         European Starling       0.0                         7.0              0.0
           California Gull       0.0                         6.0              0.0
  Pacific-slope Flycatcher       0.0                         5.0              1.0
             Vagrant Shrew       0.0                         4.0              1.0
     American Herring Gull       0.0                         4.0     

## Average number of days of care by Animal Common Name

In [25]:
released_df = df[df["patients_disposition"] == "Released"]

# Drop missing or invalid values
released_df = released_df.dropna(subset=["patients_days_in_care", "patients_common_name"])
released_df["patients_days_in_care"] = pd.to_numeric(released_df["patients_days_in_care"], errors='coerce')

# Compute average days in care per animal type and count
avg_days = (
    released_df
    .groupby("patients_common_name")["patients_days_in_care"]
    .agg(['mean', 'count'])
    .reset_index()
)
avg_days.columns = ["patients_common_name", "patients_days_in_care", "release_count"]
avg_days = avg_days.sort_values("patients_days_in_care", ascending=True)  # Ascending for horizontal bar (longest at top)

# Plot using Plotly - horizontal bar chart
fig = px.bar(
    avg_days,
    y="patients_common_name",
    x="patients_days_in_care",
    orientation='h',
    hover_data={"patients_days_in_care": ':.2f', "release_count": ':.0f'},
    labels={
        "patients_common_name": "Animal Type", 
        "patients_days_in_care": "Average Days in Care",
        "release_count": "Total Released"
    },
    title="Average Days in Care per Animal Type (Released Only, 2014-2024)",
    height=max(400, len(avg_days) * 25)  # Dynamic height based on number of species
)

# Customize hover template
fig.update_traces(
    hovertemplate='<b>%{y}</b><br>' +
                  'Average Days in Care: %{x:.2f}<br>' +
                  'Total Released: %{customdata[0]:.0f}' +
                  '<extra></extra>'
)

fig.update_layout(
    yaxis={'categoryorder':'total ascending'},  # Ensures bars are sorted by value
    margin=dict(l=200),  # More left margin for animal names
    hoverlabel=dict(
        bgcolor="white",  # White background
        font_size=14,
        font_family="Arial",
        font_color="black",  # Black text
        bordercolor="black"  # Black border for definition
    )
)
fig.show()
fig.write_html("14to24/days_before_release_14to24.html")